# DataScience 2000B / Integrated Science 2000B Final 2024 - Open Book part

## Your Student ID: ########

## General 
The instruction for the final exam for DS2000B / IS2002B is included in this Jupyter Notebook. 

- You are allowed to use any document and source and look up documents on the internet. You are not allowed to use chatGPT, copilot, or any other AI engine that provides programming assistance. 
- You or not allowed to share documents, or communicate in any other way with people during period of final. Given that there are students with extra time, this is until 7pm.  
- You are only allowed to use the python packages listed under "preliminaries" - the use of other regression (e.g., statsmodels, numpy polyfit) or machine learning toolboxes (e.g., sklearn) is not permitted. 
- All the code you are using from previous assignments or labs needs to be included in the notebook. 
- Most questions also require some written answer. The answer to these questions should be given in full English sentences. 
- All Figures should be appropriately labeled in x and y axis.  
- The Final exam needs to be submitted on Gradescope (Final) before 5:30pm. If you have approved accommodation, you need to submit after 3.5 hrs + your extra time after the start of the exam at 2pm.
- Ensure that your notebook runs before submitting. Submit the notebook with all output included. 
- Any final submitted later than the alloted time will be scored with 0 pts.  
- **It is your responsibility that you submit the correct file. Please check that you uploaded the correct file by downloading the submitted version and opening it in jupyter before you leave the exam room.** 

## Problem description 

About 1\% of the population will be diagnosed with schizophrenia at some point in their life, usually at the age of 17-22years, with devastating consequences for the individual and their family. If we could use brain imaging and a mathematical model to predict who will develop schizophrenia, we could intervene early and potentially prevent the onset of the disease.

One brain structure that has been associated with schizophrenia is the cerebellum. The cerebellum consists of regions that are involved in motor control and in cognitive functions. The dataset for the final contains the volume of these two part of the cerebellum in 200 individuals. The individuals for this studied were sampled from a cohort of individuals with a family history of schizophrenia, and thus have a much higher risk of schizophrenia than the general population.

The ages at which the brain scans were taken varies from 5-17 (before the potential onset of the disease). The brain still undergoes considerable growth during this period. We can determine whether a child is developing normal by comparing the volumes to sex and age matched norms. In this final you will build such a normative model, and then use it to predict the risk of schizophrenia.

The dataset is provided in the file `brain_volumes.csv`. The file contains the following the following columns:
- `age`: the age of the individual in years
- `sex`: the biological sex of the individual (either `male` or `female`)
- `income`: the income of individual's family (Socioeconomic status) (either `low`, `middle`, or `high`)
- `schizophrenia`: a binary variable indicating whether the individual developed schizophrenia (1) or not (0)
- `vol_motor`: the volume of the motor regions of the cerebellum in cm^3
- `vol_cognitive`: the volume of the cognitive regions of the cerebellum in cm^3
- `z_motor`: the z-score of the motor regions' volume (relative to the normal population)
- `z_cognitive`: the z-score of the cognitive regions' volume (relative to the normal population)


In [ ]:
# Preliminaries - you are only allowed to import the following packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as so

## Task 1: (8pts)
### Questions 1.1: (2pts)
Report the numbers of children that develop schizophrenia (and the numbers that do not) for each level of income status of the family (low, middle, high).   

From these numbers report the 
* conditional probability of developing schizophrenia given that the family has a low income level.
* conditional probability of developing schizophrenia given that the family has a high income level.

In [ ]:
D=pd.read_csv('brain_volumes.csv')
table = pd.crosstab(D.schizophrenia, D.income, margins = True)
table

In [ ]:
p_low = table['Low'][1]/table['Low']['All']
p_high = table['High'][1]/table['High']['All']
print(f'p(Schizophrenia|Low): {p_low:.3f}')
print(f'p(Schizophrenia|High): {p_high:.3f}')

*Grading: 1pt for cross-tab. 0.5 for each conditional probability.* 

### Question 1.2: (2pts)
Write a function that returns the difference in the conditional probability of the child developing Schizophrenia for **Low vs. Middle** income families. The result should be returned in percent. The function should take a dataframe as an input. A positive return value should indicate that the probability of developing Schizophrenia is higher in low-income families.   

Call the function with the original data and report the difference in probability in percent. 

In [ ]:
# Correct function (1pts) - Note that the difference could be reported the other way around
def calc_prob_diff(D):
    """Calculate the difference in conditional probability

    Args:
        D (DataFrame): recruitment data
    """
    table = pd.crosstab(D.schizophrenia, D.income, margins = True)
    p_low = table['Low'][1]/table['Low']['All']
    p_mid = table['Middle'][1]/table['Middle']['All']
    return (p_low-p_mid)*100

pdiff = calc_prob_diff(D)
# Correctly reported in percent (1pt)
print(f'Difference in probability {pdiff:.3f}')

### Question 1.3: (4pts)
Conduct a permutation test (with at least 2000 iterations) to test the directed hypothesis that the probability of developing schizophrenia is higher in low-income families than in middle-income families. 

- Clearly state the null hypothesis [1pt]
- Provide a histogram of the test statistic and indicate the observed value. [1pt]
- Report the correct p-value [1pt] 
- Draw a conclusion from the data [1pt]  

The corresponding Null-hypothesis is that the probability of developing schizophrenia for low-income family is the same or lower than for middle-income families. [1pt]

In [ ]:
# Write a function that shuffels the column
# of one single variable. We need to be carful here,
# as the functions get the data frame as an object
def randomize_column (df,colname):
    df_copy = df.copy()
    var = df_copy[colname].values
    np.random.shuffle(var)
    df_copy[colname]=var
    return df_copy

def permutation_test(data):
    numIter = 2000
    stat = np.zeros(numIter,) # initialize the numpy array that will store the test stat

    ## 1. shuffling the "exchangeable" variable
    for i in range(numIter):

        ## permute (shuffle) the exchangeable variable
        tmpDF = randomize_column(data,'schizophrenia')

        ## calculate the test stat.
        stat[i] = calc_prob_diff(tmpDF)

    ## 2. plotting the histogram of the test statistic
    plt.figure()
    plt.hist(stat, bins = 10)
    empStat = calc_prob_diff(data)
    plt.axvline(x = empStat, color = 'r')
    plt.xlabel('Difference in probability (%)')
    plt.ylabel('Frequency')
    plt.show()

    ## 3. calculating the p-value (one-sided)
    p_value = sum(stat >= empStat)/len(stat)

    print('P-value of the randomisation test is p=',p_value)

    return p_value


In [ ]:
permutation_test(D)

> *Conclusion:*
Based on the p-value (p=0.043) we can reject the Null-hypothesis that children from low-income families have equal or lower probability of developing schizophrenia, as compared to middle income families.

## Task 2: (28pts)
In this task, we will build a model that models the normal development of the human cerebellum. So, for this task, we want to restrict the dataset to the individuals that did not develop schizophrenia.

Having a model of normal healthy brain growth is important, as it allows us to identify individuals that deviate from the normal growth pattern. 

### Question 2.1: (2pts) 

Create dummy coded variables for gender (male / female) 
and whether it is a low income family (Low vs. (Middle, High)).
Use these dummy variables to report the count in each of the four categories (male/low), (male/not low), (female/low), (female/not low)


In [ ]:
D['sex_ind'] = (D.sex=='Male').astype(int)
D['income_low'] = (D.income=='Low').astype(int) # [1pt]
pd.crosstab(D.sex_ind,D.income_low) #[1pt]

### Question 2.2: (4pts)
Build a regression model to predict the volume of the cognitive regions of the cerebellum (vol_cognitive) using age as explanatory variable. 
* Use only the healthy sample (non-schizophrenic) and fit the model to the data.
* Produce a scatter plot of age (x-axis) and vol_cognitive (y-axis). 
* Plot the prediction line. 

In [ ]:
def multRegPredict(b,D,xname):
    """Model prediction function for multiple regression

    Args:
        b (array): Q+1 regression coefficients for Q variables
        D (pd.DataFrame): Data frame
        xname (list): Names of explanatory variables

    Returns:
        yp (array): predicted y-values
    """
    yp=np.ones(len(D.index))*b[0]        # Intercept
    for i in range(len(xname)):
        yp=yp+D[xname[i]]*b[i+1]         # Add each regression value
    return yp

def multRegLossRSS(b,D,y,xname):
    """Residual sums of squares loss function for multiple regression
    Args:
        b (array): Q+1 regression coefficients for Q variables
        D (pd.DataFrame): Data frame
        xname (list): Names of explanatory variables

    Returns:
        rss (float): Residual sums of squares
        deriv (array): array of derivatives of the loss function with respect to b
    """
    predY = multRegPredict(b,D,xname)
    res = y-predY
    rss = sum(res**2)
    grad=np.zeros(len(b))
    grad[0]=-2*np.sum(res)
    for i in range(len(xname)):
        grad[i+1]=-2*np.sum(D[xname[i]]*res)
    return (rss,grad)

def multRegFit(D,y,xname,b0=[]):
    """Multiple regression fitting function
    Args:
        D (pd.DataFrame): Data frame
        y (array): array of y values
        xname (list): Names of explanatory variables
        b0:
    Returns:
        R2 (float): Coefficient of determination
        b (array): array of fitted parameter values
    """
    k=len(xname)+1
    if (len(b0)!=k):
        b0=np.zeros((k,))
    RES = so.minimize(multRegLossRSS,b0,args=(D,y,xname),
                    jac=True,
                    tol=1e-3)
    if (not(RES.success)):
        print('unsuccessful fit')
        print(RES)
    b=RES.x # Results
    res = y-np.mean(y)
    TSS = sum(res**2)
    RSS,deriv = multRegLossRSS(b,D,y,xname)
    R2 = 1-RSS/TSS
    return (R2,b)

In [ ]:
Dn = D[D.schizophrenia==0]
[R2,b]=multRegFit(Dn,Dn.vol_cognitive,xname=['age'])
plt.plot(Dn.age,Dn.vol_cognitive,'.')
age=np.linspace(5,17,10)
P=pd.DataFrame({'age':age})
pred = multRegPredict(b,P,['age'])
plt.plot(age,pred,'r')
print(f'R2:{R2:.3f}')
plt.xlabel('age')
plt.ylabel('vol cognitive')

*Grading*
* Correct linear regression fit [1pt]
* Correctly limited to non-schizophrenic [1pt]
* Scatterplot and Axis labeling [1pt]
* Prediction line [1pt]

### Question 2.3 (5pts)

Build a polynomial regression model of order 2 using age as explanatory and vol_cognitive as a dependent variable. 

* As in question 2.2, only use the non-schizophrenic individuals.
* Report the R2-value. 
* Show a scatter plot of the healthy individuals and show the quadratic prediction line. 
* On this plot, add the schizophrenic individuals in a different color.
* Written answer: What do you observe? 

In [ ]:
D['age2']=D.age**2
Dn = D[D.schizophrenia==0]

[R2,b]=multRegFit(Dn,Dn.vol_cognitive,xname=['age','age2'])
plt.plot(Dn.age,Dn.vol_cognitive,'.')
age=np.linspace(5,17,10)
P=pd.DataFrame({'age':age,'age2':age**2})
pred = multRegPredict(b,P,['age','age2'])
plt.plot(age,pred,'r')
print(f'R2:{R2:.3f}')
plt.plot()

Ds = D[D.schizophrenia==1]
plt.plot(Ds.age,Ds.vol_cognitive,'ro')
plt.xlabel('age')
plt.ylabel('vol cognitive')

Even though patients seems to fall above and below the line, they seem to be mostly below (lower volume)

*Grading*
* Correct Polynomial model and R2 [1pt]
* Correctly limited to non-schizophrenic [1pt]
* Axis labeling [1pt]
* prediction line  [1pt]
* Additional points plotted + observation [1pt]

### Question 2.4 (6pts)
Extend the polynomial order 2 model from question 2.3, by adding the sex of the child as an additional regressor. 
Again, fir the model only to the healthy (non-schizophrenic) data. 

* Report the $R^2$ value of the fit.
* Show a scatter plot of the healthy individuals, plotted in different colors depending on the sex of the individual (age on the x, vol_cognitive on the y-axis)
* Add separate prediction lines for females and males. 
* Written answer: How do males and females differ in terms of the growth of their brain volume? 

*Hint: If cannot figure out how to combine a polynomial order 2 with a discrete predictor variable (sex), implement a simple linear regression model with age and sex as regressor to get partial points*

In [ ]:
[R2,b]=multRegFit(Dn,Dn.vol_cognitive,xname=['age','age2','sex_ind'])
plt.plot(Dn.age[Dn.sex_ind==0],Dn.vol_cognitive[Dn.sex_ind==0],'r.')
plt.plot(Dn.age[Dn.sex_ind==1],Dn.vol_cognitive[Dn.sex_ind==1],'b.')

age=np.linspace(5,17,10)
P0=pd.DataFrame({'age':age,'age2':age**2,'sex_ind':0})
pred = multRegPredict(b,P0,['age','age2','sex_ind'])
plt.plot(age,pred,'r')
P1=pd.DataFrame({'age':age,'age2':age**2,'sex_ind':1})
pred = multRegPredict(b,P1,['age','age2','sex_ind'])
plt.plot(age,pred,'b')

print(f'R2:{R2:.3f}')
plt.xlabel('age')
plt.ylabel('vol cognitive')

Males appear to have a larger volume in this brain structure. The growth rates, however, seem to be similar.

*Grading*
* Correct Polynomial model + Sex [3pt, 1pt for simple linear regression]. 
* Correctly limited to non-schizophrenic [1pt]
* prediction lines [1pt]
* Observation (mean difference is enough) [1pt]

### Question 2.5 (1pt) 
Assume that the model in question 2.4 yielded a higher $R^2$ value than the one in question 2.3. A researcher concludes from this observation that the volume of the structure differs between the sexes. Why can you NOT draw this conclusion from this analysis? 

The model in 2.4 includes all regressors from model 2.3 and one additional one. This means that the R2 needs to be at least as high as the $R^2$ for the one in 2.3. Thus a difference in R2 does not mean that the more complex model in 2.4 is better - it could simply overfit the data.   

### Question 2.6 (5pts)
In this question, you will test whether the volume of the cognitive part of the cerebellum differs between the sexes, after accounting for the effect of age. For this, perform a model comparison: 

* Choose the two appropriate models to compare and justify your answer.   
* Conduct a 10-fold cross-validation to compare the two models. 
* What criterion do you use to compare the models? Report the value of this criterion.   
* What is your conclusion?

*Hint: For full points, you do NOT require a measure of strength of evidence (Bayes Factor)* 

To account of the effect of age appropriately, I choose the quadratic model. To then test whether sex has an influence on top of this trend, I compare it to the $y=age+age^2+sex$ model. 

In [ ]:
def KfoldCVmultReg(D,y,xname,K=10):
    """K-fold Crossvalidation for multiple regression"""
    N = len(y) #Number of observations
    yp= np.zeros(N)

    # Make an index vector with K folds
    ind = np.arange(N)
    ind = np.floor(ind/N*K)

    # Get overall model fit
    R2,b_all=multRegFit(D,y,xname)

    # Loop over the crossvalidation folds
    for i in range(K):
        r,b=multRegFit(D[ind!=i],y[ind!=i],xname,b0=b_all)
        yp[ind==i]=multRegPredict(b,D[ind==i],xname)

    # Calculate crossvalidated model fit
    TSS  = sum((y-y.mean())**2)
    RSScv = sum((y-yp)**2)
    R2cv = 1-RSScv/TSS
    return R2cv,R2

In [ ]:
pR2_H0,R2_H0=KfoldCVmultReg(Dn,Dn.vol_cognitive,['age','age2'])
pR2_H1,R2_H1=KfoldCVmultReg(Dn,Dn.vol_cognitive,['age','age2','sex_ind'])
print(f'Sex as a predictor variable increases predictive R2 from {pR2_H0:.3f} to {pR2_H1:.3f}')

Given the increase in predictive R2, there is a substantial increase in predictive accuracy, when adding sex to the model. 
Thus sex does appear to have an influence on volume (after accounting for age). 

**GRading** 
* Pick the right models [2pt]
* Cross-validation routine [1pt]
* Predictive R2 as criterion [1pt] 
* Correct conclusion [1pt]

### Question 2.7.(5pts)
To characterize the volume of individuals in relationship to the age- and sex-appropriate comparison group, researchers use z-scores. These are calculated by determining the deviation of the the individual score against the model prediction (using the model from question 2.4), and then z-standardizing these residuals. For the z-standardization, use the mean and the standard deviation from the deviations of the non-schizophrenic control group only. 

* Calculate z-score for the deviation of vol_cognitive for all individuals in the sample. [1pt]
* Report the mean and standard deviation of the z-scores of the non-schizophrenic and schizophrenic individuals separately. [2pt]
* Written answer: Explain why you got the particular values for the non-schizophrenic individuals - and why does the same logic not apply to the schizophrenic patients? [1pt]
* Written answer: What can you conclude from the mean and standard deviation of the z-scores of the schizophrenic individuals? [1pt]

*Hint: Note that the z-scores provided in the data frame are derived from a larger normative model. So your answer will likely differ somewhat from the column `z_cognitive`.* 

In [ ]:
# Get the differences from the prediction for all the patients
pred = multRegPredict(b,D,['age','age2','sex_ind'])
residual = D.vol_cognitive - pred
m = residual[D.schizophrenia==0].mean()
sd = residual[D.schizophrenia==0].std()
D['z-score']=(residual-m)/sd
M = D.groupby('schizophrenia')['z-score'].mean()
S = D.groupby('schizophrenia')['z-score'].std()
print(f'Healthy Mean: {M[0]:.3f}  SD: {S[0]:.3f}')
print(f'Schizophrenic Mean: {M[1]:.3f}  SD: {S[1]:.3f}')

The healthy mean and standard deviation are 0 and 1, respectively. This is because the z-score is calculated by subtracting the mean and dividing by the standard deviation. This is not the case for the schizophrenic individuals, as we used the mean and sd of the healthy normal controls. 

The mean z-value indicates that patients are nearly half an sd below the healthy mean. The standard deviation indicates that the z-scores are equally variable as in the healthy group.

**Grading**
* Calculate z-scores  [1pt]
* Report the mean and standard deviation for non-schizophrenic and schizophrenic [2pt]
* Written answer: Explain mean + sd  [1pt]
* Written answer: Conclusion [1pt]


## Task 3: Logistic regression (14pts)
In this task, we will use the brain volumes of the cognitive and motor regions of the cerebellum to predict whether a teenager will develop schizophrenia or not. For this we will use the z-scores that are calculated from a larger and more representative model of the normal population (`z_motor` and `z_cognitive`). 

### Question 3.1  (3pts)
Fit logistic regression model that predicts the probability of developing schizophrenia based on sex and family income (low vs. medium/high).

Report the regression coefficients of the model. 

Written answers: 
Based on the the coefficients, answer the following questions: 
* does being male increase or decrease the probability of developing schizophrenia in the sample? 
* does being from a low-income family increase or decrease the probability of developing schizophrenia in the sample?
* which one of the two factors has a larger effect on the probability of developing schizophrenia?

Finally provide a reason why the intercept coefficient has a negative value. What can you conclude about the predicted probabilities?  

In [ ]:
def logisticRegPredict(b,D,xname):
    yp=np.ones(len(D.index))*b[0]       # Start out with the intercept
    for i in range(len(xname)):
        yp=yp+D[xname[i]]*b[i+1]        # Add the prediction of each regressor seperately
    p = np.exp(yp)/(1+np.exp(yp))
    p = p.clip(1e-12,1-(1e-12))
    return p

def logisticRegLoss(b,D,y,xname):
    p = logisticRegPredict(b,D,xname)
    cost = -y*np.log(p)-(1-y)*np.log(1-p)
    N=len(xname)
    grad=np.zeros(N+1)
    res = y-p
    grad[0]=-sum(res)
    for i in range(N):
        grad[i+1]=-np.sum(D[xname[i]]*res)         # Add each regressor
    return (cost.sum(),grad)

def logisticRegFit(D,y,xname,figure=0,b_init=[]):
    k=len(xname)+1
    if (len(b_init)!=k):
        b_init=np.zeros(k)
    RES = so.minimize(logisticRegLoss,b_init,args=(D,y,xname),jac=True)
    b = RES.x
    ll = -RES.fun # Negative function value is the log likelihood
    p = logisticRegPredict(b,D,xname)
    if (k==2 & figure==1):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.scatter(D[xname[0]],y)
        xRange=[min(D[xname[0]]),max(D[xname[0]])]
        xp=np.arange(xRange[0],xRange[1],(xRange[1]-xRange[0])/50)
        yp=b[0]+b[1]*xp
        pp=np.exp(yp)/(1+np.exp(yp))
        ax.plot(xp,pp,'r-')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    return (ll,b)

In [ ]:
ll,b=logisticRegFit(D,D.schizophrenia,xname=['sex_ind','income_low']) # ourrect model + coefficients: 1pt
print(b)

Written answer: 
* The coefficient for sex is positive, so being male slightly increases the prob of developing schizophrenia. 
* The coefficient for low income is positive, so being from a low-income family slightly increases the prob of developing schizophrenia. 
* The coefficient for income is larger than the for sex, so income has a larger effect on the probability [1pt]

The intercept is negative because the probability of developing schizophrenia is lower than 0.5 for the the average female individual from a medium/high income family. [1pt]

### Question 3.2 (6pts)
You are tasked with developing a model that predicts the chance of developing schizophrenia based on some combination of the following variables:
* sex
* income (low vs. medium/high)
* z_motor
* z_cognitive

Describe what strategy you would use to pick the best combination of predictor variables. What criterion do you use to decide which model is better? Implement your strategy and report the best model. 

*Hint: There are multiple strategies that are correct. Simply pick one, describe it in a few sentences and execute it. In the interest of time, I recommend choosing a simple strategy.*

*Note that forward stepwise regression, backwards stepwise regression, and complete model search, or using regularized regression are all correct answers here. As an example I give the forward stepwise regression.*

To select the best combination of predictors, I will use a stepwise selection strategy. I will start with the null model and add the predictor that increases the log-liklihood the most. I will continue to add predictors until the cross-validated log-likelihood decreases. For crossvalidation I will use 20-fold cross-validation.

In [ ]:
def KfoldCVlogisticReg(D,y,xname,K=20,fitfcn=logisticRegFit,predictfcn=logisticRegPredict):
    N = len(y) #Number of observations
    yp= np.zeros(N)
    ind = np.arange(N)
    ind = np.floor(ind/N*K)

    # Get overall model fit
    LL,b_all=fitfcn(D,y,xname,figure=0)

    # Loop over the crossvalidation folds
    for i in range(K):
        r,b=fitfcn(D[ind!=i],y[ind!=i],xname,b_init=b_all,figure=0)
        yp[ind==i]=predictfcn(b,D[ind==i],xname)
    LLcv = sum(y*np.log(yp)+(1-y)*np.log(1-yp))
    return LLcv,LL

In [ ]:
L0,ll = KfoldCVlogisticReg(D,D.schizophrenia,[],K=20)
L1,ll = KfoldCVlogisticReg(D,D.schizophrenia,['sex_ind'],K=20)
L2,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low'],K=20)
L3,ll = KfoldCVlogisticReg(D,D.schizophrenia,['z_motor'],K=20)
L4,ll = KfoldCVlogisticReg(D,D.schizophrenia,['z_cognitive'],K=20)
print(f'Sex: {L1-L0:.3f} Income:{L2-L0:.3f} motor:{L3-L0:.3f} cognitive:{L4-L0:.3f}')

Based on these differences, I would add income as a predictor. 

In [ ]:
L21,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','sex_ind'],K=20)
L23,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','z_motor'],K=20)
L24,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','z_cognitive'],K=20)
print(f'Sex: {L21-L2:.3f} motor:{L23-L2:.3f} cognitive:{L24-L2:.3f}')

Based on these restuls, we need to add z-scores for the cognitive regions of the cerebellum to the model

In [ ]:
L241,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','z_cognitive','sex_ind'],K=20)
L243,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','z_cognitive','z_motor'],K=20)
print(f'Sex: {L241-L24:.3f} motor:{L243-L24:.3f}')

Based on this result, we would add motor to the model

In [ ]:
L2431,ll = KfoldCVlogisticReg(D,D.schizophrenia,['income_low','z_cognitive','z_motor','sex_ind'],K=20)
print(f'Sex: {L2431-L243:.3f}')

We are not increasing the model complexity any further, as the log-likelihood is decreasing. So, my final model includes the regression 'income_low','z_cognitive', and 'z_motor'. 

**Grading:**  
* Appropriate model selection strategy described [2pts]
* Appropriate evaluation criterion chosen [1pts]
* Strategy correctly executed [2pts]
* Clear and correct conclusion [1pts]

### Question 3.3 (5pts)
Independent of your answer to question 3.2, let us assume you think that the model with z_cognitive and z_motor is the best model. 

You now want to deploy the model to help child psychiatrists identify children at risk of developing schizophrenia. To do this, your model will raise a `red flag` (predict schizophrenia) if the predicted probability that the child will later develop schizophrenia is larger than 0.5. If the probability is below 0.5, your model does not raise a `red flag` (predicts no schizophrenia). 

Using only the data that you are given, answer the following questions: 
* Estimate how accurate your model will likely be to predict schizophrenia in a completely **new** sample. 
* Estimate the sensitivity of your model (the probability of predicting schizophrenia in a person that will develop schizophrenia, again in a new sample). 
* Estimate the false alarm rate of your model (the probability of predicting schizophrenia in a person that will not develop schizophrenia, again in a new sample)?

*Hint: If you cannot figure out how to estimate these numbers for a new sample, report accuracy, sensitivity, and false alarm rate for the present sample for partial points.* 

In [ ]:
def cv_pred_logisticReg(D,y,xname,K=20,fitfcn=logisticRegFit,predictfcn=logisticRegPredict):
    N = len(y) #Number of observations
    yp= np.zeros(N)
    ind = np.arange(N)
    ind = np.floor(ind/N*K)

    # Get overall model fit
    LL,b_all=fitfcn(D,y,xname,figure=0)

    # Loop over the crossvalidation folds
    for i in range(K):
        r,b=fitfcn(D[ind!=i],y[ind!=i],xname,b_init=b_all,figure=0)
        yp[ind==i]=predictfcn(b,D[ind==i],xname)
    return yp

In [ ]:
yp = cv_pred_logisticReg(D,D.schizophrenia,['z_motor','z_cognitive'],20)

In [ ]:
decision = yp>0.5
accuracy = (decision == D.schizophrenia).mean()
print(f'Accuracy: {accuracy:.3f}')

In [ ]:
sensitivity = (decision & D.schizophrenia).sum()/D.schizophrenia.sum()
false_alarm = (decision & (D.schizophrenia==0)).sum()/(D.schizophrenia==0).sum()
print(f'sensitivity: {sensitivity:.3f}')
print(f'False_alarm: {false_alarm:.3f}')

** Grading:**
* Get the predictions from a suitable Cross-validation routine [2pts]
* Report Accuracy [1pt]
* Report Sensitivity [1pt]
* Report False alarm [1pt]

**Congrats: You are done with the Final and Data Science 2000. This was a challenging course, and if you finished the final, you can be proud of yourself!**